# This is just running detect on validation images that a model was trained on

In [17]:
import numpy as np
import pandas as pd
import os


import plotly.graph_objects as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import seaborn as sns
import matplotlib.pyplot as plt
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)
import plotly_express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from plotly.offline import init_notebook_mode
import plotly.io as pio
from plotly.subplots import make_subplots
# setting default template to plotly_white for all visualizations
pio.templates.default = "plotly_white"
%matplotlib inline
import gc

import utils_eda as uteda

import warnings
warnings.filterwarnings('ignore')
#get a list of all the files
from tqdm.auto import tqdm

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Where is data to run preds on?

In [7]:
IMAGES_PATH = '/home/keith/AA_jupyter_tuts/kaggle_SIIM_COVID_Detection/tmp/covid/images/valid'

# Load a trained yolov5 models and run predictions on files in IMAGES_PATH.


In [8]:
import os
cwd = os.path.abspath(os.getcwd())

# MODEL1_PATH_2CLASSES ='/artifacts/run_3bh5hck7_model:v199/best.pt'#use alone
exp15_weights=cwd +"/tmp/yolov5/kaggle-siim-covid/exp15/weights/best.pt" # .53 at .131
exp21_weights=cwd +"/tmp/yolov5/kaggle-siim-covid/exp21/weights/best.pt" # .52 at .152
exp20_weights=cwd +"/tmp/yolov5/kaggle-siim-covid/exp20/weights/best.pt" # .48 at .182
exp19_weights=cwd +'/tmp/yolov5/kaggle-siim-covid/exp19/weights/best.pt' # .49 at .194

print(exp15_weights)

/home/keith/AA_jupyter_tuts/kaggle_SIIM_COVID_Detection/tmp/yolov5/kaggle-siim-covid/exp15/weights/best.pt


# Run models, or ensembles of models


In [9]:
# keys= name of the predict output dir
# vals=model weights we are using per run 
run_names=['exp15','exp19','exp20','exp21']
model_weights=[ [exp15_weights,"0.131"], [exp19_weights, ".194"],[exp20_weights,".182"], [exp21_weights,".152"]]

# model_weights=[ MODEL_PATH_1CLASS + " "+MODEL_PATH_2CLASS, MODEL_PATH_1CLASS,MODEL_PATH_2CLASS, MODEL_PATH_3CLASS_KAGGLE]
model_sel=dict(zip(run_names,model_weights))
# model_sel

In [32]:
%%time
%cd tmp/yolov5

for key, val in model_sel.items():   
    %rm -rf ./runs/detect/{key}
#     params="--weights " + val+" --source " + TEST_DIR+ "  --img 512 --conf 0.281 --iou-thres 0.5 --max-det 8  --save-txt  --save-conf --name " + key + " --exist-ok"
#CHANGING CONFIDENCE SCORE
    params="--weights " + val[0]+" --source " + IMAGES_PATH+ "  --img 640 --conf " + val[1] +" --iou-thres 0.5 --max-det 4  --save-txt  --save-conf --name " + key + " --exist-ok"

    !python detect.py {params} >out.txt
    
# #get back to correct dir
%cd ../..

/home/keith/AA_jupyter_tuts/kaggle_SIIM_COVID_Detection/tmp/yolov5
YOLOv5 🚀 v5.0-294-gdd62e2d torch 1.9.0+cu102 CUDA:0 (TITAN Xp, 12194.0625MB)

Fusing layers... 
/home/keith/anaconda3/envs/p39/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)
Model Summary: 224 layers, 7053910 parameters, 0 gradients, 16.3 GFLOPs
YOLOv5 🚀 v5.0-294-gdd62e2d torch 1.9.0+cu102 CUDA:0 (TITAN Xp, 12194.0625MB)

Fusing layers... 
/home/keith/anaconda3/envs/p39/lib/python3.9/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important unti

# Outputs are in ./tmp/yolov5/runs/detect/exp15 (exp16 etc)
## labels are in ./tmp/yolov5/runs/detect/exp15/labels 
## 1st is class, last is confidence.  Looks like following
1 0.326869 0.343108 0.252468 0.463613 0.402857 <br>
1 0.695874 0.373716 0.246474 0.478596 0.450281


In [10]:
#how many files did detect run on?
fles=uteda.get_files(IMAGES_PATH)
fles

#lets get the image id
fls1=[fle.split('/')[-1] for fle in fles]
fls1=[fle.split('.')[0] for fle in fls1]

In [3]:
image_df, study_df, submission=uteda.get_all_dataframes()
image_df

,id,boxes,label,StudyInstanceUID,dim0,dim1,split
0,000a312787f2,"[{'x': 789.28836, 'y': 582.43035, 'width': 102...",opacity 1 789.28836 582.43035 1815.94498 2499....,5776db0cec75,3488,4256,train
1,000c3a3f293f,NaN,none 1 0 0 1 1,ff0879eb20ed,2320,2832,train
2,0012ff7358bc,"[{'x': 677.42216, 'y': 197.97662, 'width': 867...",opacity 1 677.42216 197.97662 1545.21983 1197....,9d514ce429a7,2544,3056,train
3,001398f4ff4f,"[{'x': 2729, 'y': 2181.33331, 'width': 948.000...",opacity 1 2729 2181.33331 3677.00012 2785.33331,28dddc8559b2,3520,4280,train
4,001bd15d1891,"[{'x': 623.23328, 'y': 1050, 'width': 714, 'he...",opacity 1 623.23328 1050 1337.23328 2156 opaci...,dfd9fdd85a3e,2800,3408,train
...,...,...,...,...,...,...,...
6329,ffcc6edd9445,NaN,none 1 0 0 1 1,7e6c68462e06,4240,3480,train
6330,ffd91a2c4ca0,NaN,none 1 0 0 1 1,8332bdaddb6e,2800,3408,train
6331,ffd9b6cf2961,"[{'x': 2197.38566, 'y': 841.07361, 'width': 31...",opacity 1 2197.38566 841.07361 2513.80265 1292...,7eed9af03814,2388,3050,train
6332,ffdc682f7680,"[{'x': 2729.27083, 'y': 332.26044, 'width': 14...",opacity 1 2729.27083 332.26044 4225.52099 2936...,a0cb0b96fb3d,3488,4256,train


# Select from image_df all rows where id is in fls1

In [11]:
df=image_df[image_df['id'].isin(fls1)]
df.reset_index(inplace=True, drop=True)

In [12]:
len(df)

634

#  Get correct bboxes

In [18]:
# get boxes
all_boxes=[]

for i in tqdm(range(len(df))):
    all_boxes.append(uteda.get_boxes(df.loc[i]))


  0%|          | 0/634 [00:00<?, ?it/s]